In [ ]:
import joblib

# Load the saved model
gb_classifier = joblib.load('/content/gb_classifier_DiabetesUnlocked_model.joblib')
print("Model loaded successfully!")

Model loaded successfully!


The code efficiently restores a trained Gradient Boosting model from storage using joblib, enabling rapid model deployment and reuse without retraining, which is ideal for production environments or follow-up analyses.

In [ ]:
def get_user_input():
    # Helper to handle valid integer input with constraints
    def get_int_input(prompt, positive_only=True):
        while True:
            try:
                value = int(input(prompt))
                if positive_only and value < 0:
                    print("Please enter a positive integer.")
                    continue
                return value
            except ValueError:
                print("Please enter a valid integer.")

    # Collect inputs with necessary conversions
    age = get_int_input("Enter the value for Age: ")
    weight = get_int_input("Enter your weight in kg: ")
    height = float(input("Enter your height in cm: "))
    physhlth = abs(get_int_input("Enter the value for PhysHlth (positive integer): ", positive_only=False))
    menthlth = abs(get_int_input("Enter the value for MentHlth (positive integer): ", positive_only=False))
    highbp = get_int_input("Enter the value for HighBP (0 or 1): ", positive_only=False)
    smoker = get_int_input("Enter the value for Smoker (0 or 1): ", positive_only=False)
    fruits = get_int_input("Enter the value for Fruits (0 or 1): ", positive_only=False)
    veggies = get_int_input("Enter the value for Veggies (0 or 1): ", positive_only=False)
    sex = get_int_input("Enter the value for Sex (0 for Male, 1 for Female): ", positive_only=False)
    highchol = get_int_input("Enter the value for HighChol (0 or 1): ", positive_only=False)

    # Calculate BMI
    bmi = weight / (height / 100) ** 2

    # Calculate BMR based on gender
    if sex == 0:  # Male
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:  # Female
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    # Return user features excluding height and weight
    return [age, bmi, bmr, physhlth, menthlth, highbp, smoker, fruits, veggies, sex, highchol]





This code defines a helper function, get_user_input(), designed to handle integer inputs with specific constraints, such as allowing only positive integers. get_user_input() Function: Main function that could collect user inputs.

get_int_input(prompt, positive_only=True): A nested helper function within get_user_input() that:

Takes a prompt message and an optional flag, positive_only, which, when set to True, restricts the input to positive integers.
Continuously prompts the user until they enter a valid integer (and a positive one if positive_only is True), using try-except to handle invalid entries gracefully.
In summary, get_user_input() is designed to gather and validate integer inputs, ensuring they meet specified conditions such as positivity, making it robust for constrained integer data collection.








In [ ]:
def suggest_health_advice(prediction, user_features):
    # Extract only the first 10 elements from user_features
    bmi, age, phys_hlth, ment_hlth, highbp, smoker, fruits, veggies, sex, highchol = user_features[:10]

    # Diet recommendations
    if prediction == 1:  # Diabetic
        print("Diet Recommendation: High in fiber, low in sugar and refined carbs.")
        if fruits == 0:
            print("- Increase fruit intake, focusing on low-glycemic fruits like berries.")
        if veggies == 0:
            print("- Increase intake of leafy greens and non-starchy vegetables.")
        if highchol == 1:
            print("- Limit saturated fats and include foods like nuts, seeds, and fish for healthy fats.")
        if smoker == 1:
            print("- Consider quitting smoking to reduce health risks.")
    else:  # Non-Diabetic
        print("Diet Recommendation: Balanced diet focusing on whole foods.")
        if fruits == 0 or veggies == 0:
            print("- Include more fruits and vegetables daily.")
        if highchol == 1:
            print("- Watch saturated fats and add fiber-rich foods.")
        if smoker == 1:
            print("- Reducing or quitting smoking can improve overall health.")

    # Exercise recommendations
    suggest_exercise(bmi, age, phys_hlth, ment_hlth, highbp, smoker, sex, highchol)

def suggest_exercise(bmi, age, phys_hlth, ment_hlth, highbp, smoker, sex, highchol):
    # Validate inputs to ensure they are within expected ranges
    highbp = 1 if highbp > 0 else 0  # Assume any positive value as high blood pressure
    smoker = 1 if smoker > 0 else 0  # Assume any positive value as a smoker

    # Basic exercise suggestions based on criteria
    if bmi < 18.5:
        exercise = "Consider weight-gaining exercises like resistance training to build muscle mass."
    elif 18.5 <= bmi < 24.9:
        exercise = "Engage in regular cardiovascular exercises such as walking, jogging, or cycling."
    elif 25 <= bmi < 29.9:
        exercise = "Incorporate moderate-intensity exercises like swimming or brisk walking into your routine."
    else:  # BMI >= 30
        exercise = "Focus on low-impact exercises such as water aerobics or chair exercises to start."

    if age > 60:
        exercise += " Also, consider gentle stretching and balance exercises to prevent falls."

    if highbp == 1:
        exercise += " Monitor your heart rate during exercises, and consult with a physician for specific guidelines."

    if smoker == 1:
        exercise += " It's important to quit smoking and incorporate lung-strengthening exercises like walking or yoga."

    if highchol == 1:
        exercise += " Include cardiovascular exercises to help manage cholesterol levels."

    print("Exercise Recommendation:", exercise)




The code provides personalized health advice, with diet and exercise recommendations based on an individual's diabetic prediction and health metrics.

Key Functions:
suggest_health_advice(prediction, user_features):
Generates dietary advice based on the diabetes prediction (1 for diabetic, 0 for non-diabetic).
Calls suggest_exercise to provide tailored exercise tips.
suggest_exercise(bmi, age, phys_hlth, ment_hlth, highbp, smoker, sex, highchol):
Recommends exercise routines based on BMI, age, and health indicators (e.g., blood pressure, smoking status).
Special cases (e.g., older age or high cholesterol) add specific advice for improved health outcomes.
Purpose:
This structure ensures advice is personalized to support healthier lifestyle choices aligned with an individual's health status and habits.

In [ ]:
# Collect user input
user_features = get_user_input()

# Prepare features for the model (this excludes height and weight)
features_for_model = user_features[:2] + user_features[3:]  # Exclude height and weight

# Make prediction
prediction = gb_classifier.predict([features_for_model])[0]
if prediction == 1:
    print("The individual is predicted to be diabetic.")
else:
    print("The individual is predicted to be non-diabetic.")

Enter the value for Age: 32
Enter your weight in kg: 60
Enter your height in cm: 170
Enter the value for PhysHlth (positive integer): 15
Enter the value for MentHlth (positive integer): 16
Enter the value for HighBP (0 or 1): 0
Enter the value for Smoker (0 or 1): 0
Enter the value for Fruits (0 or 1): 1
Enter the value for Veggies (0 or 1): 1
Enter the value for Sex (0 for Male, 1 for Female): 1
Enter the value for HighChol (0 or 1): 0
The individual is predicted to be non-diabetic.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


This code collects user health data, prepares it by removing height and weight (as BMI is calculated separately), and uses a pre-trained diabetes model to make a prediction. Based on the model’s output, it displays whether the user is likely diabetic or non-diabetic.

In [ ]:
# Call the combined suggestion function
suggest_health_advice(prediction, user_features)

Diet Recommendation: Balanced diet focusing on whole foods.
- Include more fruits and vegetables daily.
- Watch saturated fats and add fiber-rich foods.
Exercise Recommendation: Focus on low-impact exercises such as water aerobics or chair exercises to start. Monitor your heart rate during exercises, and consult with a physician for specific guidelines. Include cardiovascular exercises to help manage cholesterol levels.


This line calls the suggest_health_advice function with the diabetes prediction and user’s health data as input. It provides personalized diet and exercise recommendations based on the prediction result and individual health attributes.

In conclusion, this code provides a comprehensive approach to predicting diabetes risk and offering tailored health advice. After gathering user-specific data, it uses a trained model to predict diabetes status. Based on this prediction and the user’s health metrics, it generates customized diet and exercise recommendations, helping individuals make informed lifestyle adjustments to improve or maintain their health. This model combines predictive insights with actionable guidance, making it a practical tool for health management.